필요한 라이브러리 설치 및 데이터 불러오기


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install konlpy


In [105]:
import numpy as np
import pandas as pd
import csv

import matplotlib.pyplot as plt
import seaborn as sns

from konlpy.tag import *
from collections import Counter
from transformers import AutoModel, AutoTokenizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf
import torch

테스트 예시


In [139]:
# 파일 가져오기
df_file = "C:/Users/pc/Desktop/민지/동아리/프로젝트(13기)/최종_데이터셋7(월,일,색상,RGB).csv"
stopword_file = "/content/drive/MyDrive/TAVE프로젝트/data/fire.txt"

df = pd.read_csv(df_file) #, encoding='CP949'
df.drop(["일자", "선택한 색상", "중앙 색상들"], axis=1, inplace=True)


NLP 처리 및 데이터 전처리


In [140]:
# NLP 처리 할 데이터만 가져오기
df_nlp = df[["꽃", "꽃말", "설명", "색상"]]
print(df_nlp.shape)
df_nlp.head(5)

# 꽃말, 설명 합치기 (색상은 유지)
df_nlp['설명'] = df_nlp['꽃말'] + ". " + df_nlp['설명']

# 특수문자 및 띄어쓰기 제거 함수
import re

def remove_special_characters(text):
    if not isinstance(text, str):
        text = str(text)

    # 특수문자 제거
    pattern = r'[^\w\s\.]' # 문자, 공백문자, 마침표 제외 제거
    clean_text = re.sub(pattern, '', text)

    # 한자 제거
    pattern = r'[\u4e00-\u9fff]' # 중국어 한자의 유니코드 시작과 끝 제거
    clean_text = re.sub(pattern, '', clean_text)
    clean_text = ' '.join(clean_text.split())
    return clean_text

df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)
df_nlp['색상'] = df_nlp['색상'].apply(remove_special_characters)

(837, 4)


C:\Users\pc\AppData\Local\Temp\ipykernel_10428\2385004683.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['꽃말'] + ". " + df_nlp['설명']
C:\Users\pc\AppData\Local\Temp\ipykernel_10428\2385004683.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명'] = df_nlp['설명'].apply(remove_special_characters)
C:\Users\pc\AppData\Local\Temp\ipykernel_10428\2385004683.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

Klue-RoBERTa 모델 및 토크나이저 불러오기


In [141]:
# Klue-RoBERTa 모델과 토크나이저 불러오기
model_name = 'klue/roberta-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# [CLS]와 [SEP] 토큰을 추가하는 함수
def add_special_tokens(text):
    # 문장을 마침표를 기준으로 분리
    sentences = text.split('. ')
    # 각 문장에 [CLS]와 [SEP] 토큰 추가
    sentences_with_special_tokens = ['[CLS] ' + sentence + ' [SEP]' for sentence in sentences]
    # 다시 문장으로 결합
    text_with_special_tokens = ' '.join(sentences_with_special_tokens)
    return text_with_special_tokens

df_nlp['설명2'] = df_nlp['설명'].apply(add_special_tokens)
df_nlp['색상2'] = df_nlp['색상'].apply(add_special_tokens)
df_nlp.head(10)


Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\pc\AppData\Local\Temp\ipykernel_10428\3946500010.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nlp['설명2'] = df_nlp['설명'].apply(add_special_tokens)
C:\Users\pc\AppData\Local\Temp\ipykernel_10428\3946500010.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

,꽃,꽃말,설명,색상,설명2,색상2
0,각시붓꽃,"부끄러움, 세련됨",부끄러움 세련됨. 각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 ...,보라,[CLS] 부끄러움 세련됨 [SEP] [CLS] 각시라 하면 이제 막 시집 온 새색...,[CLS] 보라 [SEP]
1,감국,그윽한 향기,그윽한 향기. 가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인...,노랑,[CLS] 그윽한 향기 [SEP] [CLS] 가을 산야는 국화과 식물들 차지다 [S...,[CLS] 노랑 [SEP]
2,개나리,"희망,깊은 정, 달성",희망깊은 정 달성. 우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃 개...,노랑,[CLS] 희망깊은 정 달성 [SEP] [CLS] 우리나라 전역에서 봄 소식을 가장...,[CLS] 노랑 [SEP]
3,개나리,희망,희망. 개나리 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. 나리나리 개나리...,노랑,[CLS] 희망 [SEP] [CLS] 개나리 봄을 알리는 전령사 하면 가장 먼저 떠...,[CLS] 노랑 [SEP]
4,갯개미취,추억,추억. 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 ...,보라,[CLS] 추억 [SEP] [CLS] 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가...,[CLS] 보라 [SEP]
5,거베라,"신비, 수수께끼",신비 수수께끼. 거베라만큼 크고 화려한 꽃도 드물다. 부를 상징하기도 한다. 거베라...,분홍,[CLS] 신비 수수께끼 [SEP] [CLS] 거베라만큼 크고 화려한 꽃도 드물다 ...,[CLS] 분홍 [SEP]
6,검은포플라,용기,용기. 당신은 용기가 있어 주위 사람들도 당신을 의지하고 있습니다. 그렇다고 해서 ...,빨강,[CLS] 용기 [SEP] [CLS] 당신은 용기가 있어 주위 사람들도 당신을 의지...,[CLS] 빨강 [SEP]
7,겹 공작초,첫눈에 반함,첫눈에 반함. 자기가 가장 아름답다고 믿는 세 여신이 있었다. 제우스의 아내 헤라 ...,보라,[CLS] 첫눈에 반함 [SEP] [CLS] 자기가 가장 아름답다고 믿는 세 여신이...,[CLS] 보라 [SEP]
8,겹 과꽃,나의 사랑은 당신보다깊다,나의 사랑은 당신보다깊다. 괴테의 희곡 파우스트 에 등장하는 소녀 마가렛이 꽃잎을 ...,보라,[CLS] 나의 사랑은 당신보다깊다 [SEP] [CLS] 괴테의 희곡 파우스트 에 ...,[CLS] 보라 [SEP]
9,겹 소국,청초,청초. 국화 품종 중 작은 화형을 가진 국화를 소국이라고 하며 스프레이 형태를 띄고...,분홍,[CLS] 청초 [SEP] [CLS] 국화 품종 중 작은 화형을 가진 국화를 소국이...,[CLS] 분홍 [SEP]


벡터화 함수


In [142]:
# 문장을 벡터화 변환하는 함수
def get_sentence_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True) # 최대 길이 초과 시 잘라내기, 작은 경우 패딩 진행
    outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].detach().numpy() # 각 토큰 벡터의 첫 번째 벡터 확인 (CLS 토큰 벡터)

# 설명 벡터화
df_nlp['설명_벡터'] = df_nlp['설명2'].apply(lambda x: get_sentence_embedding(x, tokenizer, model).tolist()) # 데이터마다 (1, 768) 차원 벡터 생성
df_nlp['색상_벡터'] = df_nlp['색상2'].apply(lambda x: get_sentence_embedding(x, tokenizer, model).tolist())


월/계절 원핫인코딩


In [143]:
# 월/계절 원핫인코딩
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(df[['월', '계절']]).toarray()  # '월'과 '계절' 컬럼을 원핫인코딩
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['월', '계절'])) # 원핫인코딩된 결과를 새로운 데이터프레임으로 변환

# 원핫인코딩 결과를 '원핫인코딩' 컬럼으로 추가
df_nlp['원핫인코딩'] = encoded_df.apply(lambda row: list(row), axis=1)
df_nlp['원핫인코딩'] = df_nlp['원핫인코딩'].apply(lambda x: np.array(x).reshape(1, -1)) #(16,1)을 (1,16)차원으로 변경


결합 벡터 생성 함수


In [144]:
# 결합 벡터 생성 (설명 벡터, 색상 벡터, 원핫인코딩 벡터 결합)
def create_combined_vector(row):
    설명_벡터 = row['설명_벡터']
    색상_벡터 = row['색상_벡터']
    원핫인코딩 = row['원핫인코딩']

    combined_array = np.concatenate((설명_벡터, 색상_벡터, 원핫인코딩), axis=1)

    return combined_array

# df_nlp['최종_벡터'] = df_nlp.apply(create_combined_vector, axis=1)

# 색상이 없는 경우 결합 벡터 생성 함수
def create_combined_vector_without_color(row):
    설명_벡터 = row['설명_벡터']
    원핫인코딩 = row['원핫인코딩']
    combined_array = np.concatenate((설명_벡터, 원핫인코딩), axis=1)
    return combined_array


In [16]:
# #추천시스템_데이터 저장
# import json

# df_nlp['월'] = df['월']
# df_nlp['계절'] = df['계절']
# df_nlp['색상'] = df['색상']

# # 배열을 JSON 문자열로 변환
# df_nlp['설명_벡터'] = df_nlp['설명_벡터'].apply(lambda x: json.dumps(x))
# df_nlp['색상_벡터'] = df_nlp['색상_벡터'].apply(lambda x: json.dumps(x))

# df_nlp = df_nlp.reindex(columns=['꽃', '월', '계절', '꽃말', '색상', '설명', '설명_벡터', '색상_벡터', '원핫인코딩'])

# # CSV 파일로 저장
# df_nlp.to_csv("C:/Users/pc/Model/Model/dataset/추천시스템_데이터.csv", index=False)

사용자 입력 텍스트 분석 함수


In [145]:
# 사용자 입력 텍스트 분석 함수(월, 계절)
def extract_month_season(text):
    months = {
        '1월': 1, '2월': 2, '3월': 3, '4월': 4, '5월': 5, '6월': 6,
        '7월': 7, '8월': 8, '9월': 9, '10월': 10, '11월': 11, '12월': 12
    }
    seasons = {'봄': '봄', '여름': '여름', '가을': '가을', '겨울': '겨울'}

    month = None
    season = None

    for key, value in months.items():
        if key in text:
            month = value
            break

    for key in seasons.keys():
        if key in text:
            season = key
            break

    return month, season

In [146]:
# 확장된 색상 사전
color_synonyms = {
    '갈색': ['갈색', '브라운', '갈색의', '브라운색', '갈색이', '갈색이다', '갈색을', '갈색으로'],
    '노랑': ['노랑', '노란', '노란색', '황색', '노랑색', '노랗다', '노랗게', '노란빛', '노란 빛','누런'],
    '보라': ['보라', '보라색', '자주색', '보라빛', '보랏빛', '보라빛의', '보라빛이', '보라빛으로'],
    '분홍': ['분홍', '핑크', '분홍색', '핑크색', '분홍빛', '분홍빛의', '분홍빛이', '분홍빛으로'],
    '빨강': ['빨강', '빨간', '빨강색', '빨간색', '붉은', '붉은색', '붉은 빛', '붉다', '붉게'],
    '주황': ['주황', '주황색', '오렌지', '오렌지색', '주황빛', '주황빛의', '주황빛이', '주황빛으로'],
    '초록': ['초록', '초록색', '녹색', '초록빛', '초록의', '초록이', '초록으로'],
    '파랑': ['파랑', '파란', '파란색', '파랑색', '청색', '파랑빛', '파란빛', '파랗다', '파랗게','푸른','푸른빛', '푸른 색'],
    '흰색': ['흰색', '하양', '하얀', '백색', '하얀색', '하양색', '하얗다', '하얗게', '백색의', '백색이', '백색으로','흰']
}

# 입력 텍스트에서 색상을 추출하는 함수
def extract_color(text):
    text = text.lower()
    for color, synonyms in color_synonyms.items():
        for synonym in synonyms:
            if synonym in text:
                return color
    return None

In [204]:
# 형태소 분석기 초기화
okt = Okt()

# 단어 변경 (데이터셋에 있는 것으로)

def lexicon_replace(user_input):
    lexicon = {
        "연인": [
            "여자친구", "여친", "내 여자", "내 애인", "그녀", "여친님", "여사친", "여자 친구",
            "남자친구", "남친", "내 남자", "남친님", "남사친", "남자 친구", "애인",
            "사랑하는 사람", "연애 상대", "연인", "와이프"
        ],
        "졸업": [
             "졸업식", "졸업 하는 날", "학위수여식", "학위식"
        ],
        "스승": [
            "선생님", "선생", "교사", "스승님", "담임", "교원", "선생님들", "선생 님","선생님"
        ],
        "어머니": [
            "엄마", "가족", "아버지", "어머님", "아버님", "아빠"
        ]
    }
    
    noun_list = []
    pos_tags = okt.pos(user_input) 
    for word, tag in pos_tags: #형태소 분석 -> 명사만 추출 -> 단어 변경
        if tag == 'Noun':
            noun_list.append(word)
    
    for noun in noun_list:
        for key, values in lexicon.items():
            if noun in values:
                user_input = re.sub(noun, key, user_input)

    if "연인" in user_input:
        user_input += " 사랑"
    if "스승" in user_input:
        user_input += " 존경"
    if "어머니" in user_input:
        user_input += " 감사"

    return user_input

In [205]:
#점 추가
def add_dot(user_input):
    if not user_input.endswith("."):
        user_input += "."
    return user_input

형태소 확인

In [206]:
# 형태소 분석기 초기화
okt = Okt()

# 입력 텍스트의 형태소 분석 및 명사형 확인 함수
def is_noun_phrase(text):
    # 형태소 분석
    pos_tags = okt.pos(text)
    ## 디버깅을 위해 형태소 분석 결과 출력
    #print(f"Text: {text}, POS Tags: {pos_tags}")
    # 모든 단어가 명사인지 확인
    for word, tag in pos_tags:
        if tag != 'Noun':
            return False
    return True

# 명사형 텍스트에 문맥 추가 함수
def add_context_if_noun(user_input):
    if is_noun_phrase(user_input):
        return user_input + "에 어울리는 꽃을 추천해." # " 꽃을 추천해"
    return user_input

사용자 입력을 벡터화 하는 함수


In [222]:
# 사용자 입력을 최종 벡터화
def get_user_input_vector(user_input, user_month):
    # 렉시콘 변환
    user_input = lexicon_replace(user_input) 
    user_input = add_dot(user_input)
    user_input = add_context_if_noun(user_input)
    input_text = remove_special_characters(user_input)

    # 토큰화 및 벡터화
    input_embeddings = get_sentence_embedding(input_text, tokenizer, model)
    input_color_text = extract_color(input_text)
    month, season = extract_month_season(input_text)

    # 원핫 벡터 생성
    user_onehot_vector = np.zeros(len(encoder.get_feature_names_out(['월', '계절'])))
    if month is not None:
        month = user_month
        month_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'월_{month}')
        user_onehot_vector[month_idx] = 1
    if season is None and month is not None: #사용자가 입력한 월 기준으로 계절 추출
        if month in [3, 4, 5]:
            season = '봄'
        elif month in [6, 7, 8]:
            season = '여름'
        elif month in [9, 10, 11]:
            season = '가을'
        else:
            season = '겨울'
        season_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'계절_{season}')
        user_onehot_vector[season_idx] = 1

    if month is not None: #텍스트에 입력된 월
        month_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'월_{month}')
        user_onehot_vector[month_idx] = 1
    if season is not None: #텍스트에 입력된 계절
        season_idx = encoder.get_feature_names_out(['월', '계절']).tolist().index(f'계절_{season}')
        user_onehot_vector[season_idx] = 1
    user_onehot_vector = user_onehot_vector.reshape(1, -1) #(1,16)

    # 벡터 결합
    if input_color_text:
        color_embeddings = get_sentence_embedding(input_color_text, tokenizer, model)
        user_vector = np.concatenate((input_embeddings, color_embeddings, user_onehot_vector), axis=1)
    else:
        user_vector = np.concatenate((input_embeddings, user_onehot_vector), axis=1)

    return user_vector

특정 이벤트에 가중치를 부여하는 함수


In [223]:
# 특정 이벤트에 가중치를 부여하는 함수 (각 행마다 개별 적용)
def apply_event_weight_for_row(user_input, row):
    user_input = lexicon_replace(user_input) 
    
    event_weights = {
        '발렌타인': 1.2,
        '화이트': 1.2,
        '어버이날': 1.2,
        '어버이': 1.2,
        '부모': 1.2,
        '부모님': 1.2,
        '성년의날': 1.2,
        '성년': 1.2,
        '로즈데이': 1.2,
        '로즈': 1.2,
        '스승의날': 1.2,
        '선생': 1.2,
        '스승': 1.2,
        '선생님': 1.2,
        '연인': 1.2,
        '생일': 1.2,
        '기념일': 1.2,
        '사랑': 1.2,
        '졸업':1.2
    }
    weight = 1.0
    for event, event_weight in event_weights.items():
        if event in user_input:
            weight = event_weight
            break

    # 월 가중치 적용
    month, _ = extract_month_season(user_input)
    if month is not None:
        weight *= 1.2

    # 계절 가중치 적용
    season, _ = extract_month_season(user_input)
    if season is not None:
        weight *= 1.2

    # 꽃의 설명2에 따른 추가 가중치 적용
    if any(event in row['설명2'] for event in event_weights.keys() if event in user_input):
        weight *= 1.2
    # 월에 따른 추가 가중치 적용
    if month and isinstance(row['월'], int) and month == row['월']:
        weight *= 1.2
    # 계절에 따른 추가 가중치 적용
    if season and isinstance(row['계절'], str) and season == row['계절']:
        weight *= 1.2

    return weight

In [224]:
# 사용자 입력과 KLUE 모델 유사도 구하기
def Klue_similarities(user_input, user_month=None):
    user_vector = get_user_input_vector(user_input, user_month)
    input_color = extract_color(user_input)
    df = df_nlp

    if input_color:
        # 색상이 명시된 경우 해당 색상의 꽃들로 필터링
        filtered_df = df[df['색상'] == input_color]
        filtered_df['최종_벡터'] = filtered_df.apply(lambda row: create_combined_vector(row), axis=1)
    else:
        # 색상이 명시되지 않은 경우 결합 벡터에서 색상 벡터를 제외
        filtered_df = df.copy()
        filtered_df['최종_벡터'] = filtered_df.apply(lambda row: create_combined_vector_without_color(row), axis=1)

    # 각 행에 대해 가중치 계산 및 유사도 산출
    filtered_df['유사도'] = filtered_df.apply(lambda row: cosine_similarity(user_vector, np.array(row['최종_벡터']).reshape(1, -1))[0][0] * apply_event_weight_for_row(user_input, row), axis=1)
    filtered_df['유사도'] = filtered_df['유사도'].astype(float)  # 숫자형으로 변환

    return filtered_df['유사도']

In [191]:
user_input = "여자친구와 1주년이야"

In [192]:
df_nlp['model_similarity'] = Klue_similarities(user_input , 7)
df_nlp.head(5)

,꽃,꽃말,설명,색상,설명2,색상2,설명_벡터,색상_벡터,원핫인코딩,klue_similarity,tfidf_similarity,model_similarity,combined_similarity
0,각시붓꽃,"부끄러움, 세련됨",부끄러움 세련됨. 각시라 하면 이제 막 시집 온 새색시를 연상케 한다. 그래서인지 ...,보라,[CLS] 부끄러움 세련됨 [SEP] [CLS] 각시라 하면 이제 막 시집 온 새색...,[CLS] 보라 [SEP],"[[0.07368195801973343, 0.15451766550540924, 0....","[[0.15296272933483124, 0.3076050877571106, -0....","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.877557,0.0,0.878938,0.221724
1,감국,그윽한 향기,그윽한 향기. 가을 산야는 국화과 식물들 차지다. 특히 노란 꽃으로 향기까지 일품인...,노랑,[CLS] 그윽한 향기 [SEP] [CLS] 가을 산야는 국화과 식물들 차지다 [S...,[CLS] 노랑 [SEP],"[[0.08965592086315155, 0.2825712561607361, 0.1...","[[0.11612437665462494, 0.3139854669570923, -0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.874542,0.0,0.870351,0.204698
2,개나리,"희망,깊은 정, 달성",희망깊은 정 달성. 우리나라 전역에서 봄 소식을 가장 먼저 알려주는 대표적인 꽃 개...,노랑,[CLS] 희망깊은 정 달성 [SEP] [CLS] 우리나라 전역에서 봄 소식을 가장...,[CLS] 노랑 [SEP],"[[0.13718879222869873, 0.23945458233356476, 0....","[[0.11612437665462494, 0.3139854669570923, -0....","[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.888367,0.0,0.889008,0.285720
3,개나리,희망,희망. 개나리 봄을 알리는 전령사 하면 가장 먼저 떠오르는 꽃이다. 나리나리 개나리...,노랑,[CLS] 희망 [SEP] [CLS] 개나리 봄을 알리는 전령사 하면 가장 먼저 떠...,[CLS] 노랑 [SEP],"[[-0.06287881731987, 0.3564964234828949, 0.256...","[[0.11612437665462494, 0.3139854669570923, -0....","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.862052,0.0,0.859821,0.075367
4,갯개미취,추억,추억. 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가운데서도 당신은 옛 일을 어제 ...,보라,[CLS] 추억 [SEP] [CLS] 옛날 일 따위는 깨끗이 잊는 사람들이 많은 가...,[CLS] 보라 [SEP],"[[0.17346309125423431, 0.0834614634513855, 0.0...","[[0.15296272933483124, 0.3076050877571106, -0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",0.931048,0.0,0.934319,0.651421


### tfidf 적용
- add context 적용하지 않고 user_input 그대로 input들어가야함
- 텍스트 변환없이 그대로 사용 (점 추가, 명사 문장화, 월 추출 등 제외)

추천 시스템 함수


In [225]:
# 추천 시스템 함수 (코사인 유사도 기반)
def recommend_flower(user_input, user_month=None):
    df_nlp['model_similarity'] = Klue_similarities(user_input,user_month)

    # TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(df_nlp['설명']) # 문서 집합을 tfidf 벡터화
    user_tfidf = tfidf_vectorizer.transform([user_input]) # 사용자 입력 TF-IDF 벡터화
    # 각 문서와 사용자 입력 간의 코사인 유사도 계산
    similarities = cosine_similarity(user_tfidf, tfidf_matrix)
    similarities = pd.DataFrame(similarities.reshape(-1,1))
    df_nlp['tfidf_similarity'] = similarities.astype(float) 
        
    # 정규화 (0-1 범위로)
    semantic_similarities = (df_nlp['model_similarity'] - df_nlp['model_similarity'].min()) / (df_nlp['model_similarity'].max() - df_nlp['model_similarity'].min())
    tfidf_similarities = (df_nlp['tfidf_similarity'] - df_nlp['tfidf_similarity'].min()) / (df_nlp['tfidf_similarity'].max() - df_nlp['tfidf_similarity'].min())
    tfidf_similarities = tfidf_similarities.fillna(0) # NaN 값을 0으로 대체

    # 가중치 적용 및 결합
    semantic_weight=0.8
    keyword_weight = 1 - semantic_weight
    combined_scores = (semantic_weight * semantic_similarities +
                        keyword_weight * tfidf_similarities)
    df_nlp['combined_similarity'] = combined_scores
    
    # 유사도를 기준으로 상위 3개의 꽃을 선택하고 중복된 꽃을 제거
    top3 = df_nlp.nlargest(3, 'combined_similarity').drop_duplicates(subset='꽃')

    # 만약 중복 제거 후 3개의 꽃이 되지 않는 경우, 다시 nlargest로 채우기
    if top3.shape[0] < 3:
        additional_top = df_nlp.nlargest(20, 'combined_similarity')  # 상위 20개 정도를 선택
        additional_top = additional_top[~additional_top['꽃'].isin(top3['꽃'])]
        top3 = pd.concat([top3, additional_top]).nlargest(3, '유사도').drop_duplicates(subset='꽃')

    return top3[['꽃', '꽃말', 'combined_similarity']].to_dict('records')

In [226]:
user_input = "여자친구와 기념일이야"
recommend_flower(user_input, 7)

[{'꽃': '아도니스', '꽃말': '영원한 행복', 'combined_similarity': 0.8},
 {'꽃': '협죽도', '꽃말': '위험', 'combined_similarity': 0.7927276422966132},
 {'꽃': '금작화', '꽃말': '박애', 'combined_similarity': 0.7875043640175083}]

In [229]:
user_input = "연인와 기념일이야"
recommend_flower(user_input, 7)

[{'꽃': '아도니스', '꽃말': '영원한 행복', 'combined_similarity': 0.8},
 {'꽃': '협죽도', '꽃말': '위험', 'combined_similarity': 0.7927276422966132},
 {'꽃': '금작화', '꽃말': '박애', 'combined_similarity': 0.7875043640175083}]